### RESILIENT DISTRIBUTED DATASETS

### To working with RDD use SparkContext

### Run transforamtions before the actions

In [1]:
import re
from pyspark import SparkConf, SparkContext


conf = SparkConf().setMaster('local[*]').setAppName('spark-rdd-tut')
sc = SparkContext(conf=conf)

your 131072x1 screen size is bogus. expect trouble


23/03/25 18:07:14 WARN Utils: Your hostname, ProBook resolves to a loopback address: 127.0.1.1; using 172.22.34.28 instead (on interface eth0)
23/03/25 18:07:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 18:07:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
rdd_test = sc.textFile('./data/sample.txt')

In [18]:
rdd_test.collect()

['1 2 3 4 5', '3 4 5 66 77', '12 43 6 7 8', '12 12 33']

In [2]:
def parse_line(line):
    fields = line.split(',')
    station_id = fields[0]
    entry_type = fields[2]
    temperature = float(fields[3]) * 0.1 * (9/5.0) + 32.0
    temperature = (temperature - 32) * 5/9

    return(station_id, entry_type, temperature)

lines = sc.textFile("./data/1800.csv")
parsed_lines = lines.map(parse_line)
min_temps = parsed_lines.filter(lambda x: "TMIN" in x[1])
max_temps = parsed_lines.filter(lambda x: "TMAX" in x[1])
station_temps_min = min_temps.map(lambda x: (x[0], x[2]))
station_temps_max = max_temps.map(lambda x: (x[0], x[2]))
stations = station_temps_max.map(lambda x: (x[0]))
print(set(stations.collect()))
min_temps = station_temps_min.reduceByKey(lambda x,y: min(x,y))
max_temps = station_temps_max.reduceByKey(lambda x,y: max(x,y))
results_min = min_temps.collect()
results_max = max_temps.collect()

for result in results_min:
    print(f"Station {result[0]}: {round(result[1],2)}\N{DEGREE SIGN}C")

for result in results_max:
    print(f"Station {result[0]}: {round(result[1],2)}\N{DEGREE SIGN}C")


{'ITE00100554', 'EZE00100082'}


Station ITE00100554: -14.8°C
Station EZE00100082: -13.5°C
Station ITE00100554: 32.3°C
Station EZE00100082: 32.3°C


In [3]:
#conf = SparkConf().setMaster("local").setAppName("WordCount")
#sc = SparkContext(conf=conf)

input = sc.textFile('./data/book.txt')
words = input.flatMap(lambda x: x.split())
word_counts = words.countByValue()

for word, count in word_counts.items():
    clean_word = word.encode('ascii', 'ignore')
    if (clean_word):
        #print(clean_word, count)
        pass


In [4]:
import re 

In [5]:
def normalize_words(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

input = sc.textFile('./data/book.txt')
words = input.flatMap(normalize_words)
word_counts = words.countByValue()

for word, count in word_counts.items():
    clean_word = word.encode('ascii', 'ignore')
    if clean_word:
        #print(clean_word, count)
        pass

In [6]:
def normalize_words(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())


input = sc.textFile('./data/book.txt')
words = input.flatMap(normalize_words)

word_counts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

word_counts_sorted = word_counts.map(lambda x: (x[1],x[0])).sortByKey()

results = word_counts_sorted.collect()

for result in results:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if word:
        #print(f'{word}: {count}')
        pass

In [7]:
def parse_line(line):
    fields = line.split(',')
    user_id = fields[0]
    price_type = float(fields[2])
    
    return(user_id, price_type)


input_orders = sc.textFile('./data/customer-orders.csv')
orders_by_users = input_orders.map(parse_line)

orders_counts = orders_by_users.reduceByKey(lambda x, y: x + y)
sorted_orders_by_users = orders_counts.map(lambda x: (x[1], x[0])).sortByKey(ascending=False)

for i,s in enumerate(sorted_orders_by_users.collect()):
    if i < 6:
        formated = float("{:.2f}".format(s[0]))
        print(f"User: {s[1]} -> {formated}$ ")
    else:
        break


User: 68 -> 6375.45$ 
User: 73 -> 6206.2$ 
User: 39 -> 6193.11$ 
User: 54 -> 6065.39$ 
User: 71 -> 5995.66$ 
User: 2 -> 5994.59$ 


#### DATAFRAMES

### Use SparkSession when creating a dataframe

In [8]:
from pyspark.sql.types import (StructType, StructField, 
                               IntegerType, BinaryType,
                              DateType, DoubleType,
                            LongType, TimestampType)

from pyspark.sql import Row
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession \
        .builder \
        .master('local[*]') \
        .getOrCreate()

In [10]:
df_wo = spark.read.csv('./data/london_weather.csv', header=True, inferSchema=True)

In [11]:
df_wo.show(10, truncate=True)

+--------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|    date|cloud_cover|sunshine|global_radiation|max_temp|mean_temp|min_temp|precipitation|pressure|snow_depth|
+--------+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+
|19790101|        2.0|     7.0|            52.0|     2.3|     -4.1|    -7.5|          0.4|101900.0|       9.0|
|19790102|        6.0|     1.7|            27.0|     1.6|     -2.6|    -7.5|          0.0|102530.0|       8.0|
|19790103|        5.0|     0.0|            13.0|     1.3|     -2.8|    -7.2|          0.0|102050.0|       4.0|
|19790104|        8.0|     0.0|            13.0|    -0.3|     -2.6|    -6.5|          0.0|100840.0|       2.0|
|19790105|        6.0|     2.0|            29.0|     5.6|     -0.8|    -1.4|          0.0|102250.0|       1.0|
|19790106|        5.0|     3.8|            39.0|     8.3|     -0.5|    -6.6|          0.7|102780.0|       1.0|
|

In [12]:
df_wo_new = df_wo.withColumn('new_date', F.to_date(F.col("date"), 'yyyyMMdd'))

In [13]:
df_wo_new = df_wo_new.drop('date')

In [14]:
df_wo_new = df_wo_new.filter(df_wo_new.new_date >= F.lit("2017-12-01").cast(DateType())) 
df_wo_new = df_wo_new.filter(df_wo_new.new_date <= F.lit("2019-12-31").cast(DateType()))

In [15]:
df_wo_new.show(truncate=False)

+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+----------+
|cloud_cover|sunshine|global_radiation|max_temp|mean_temp|min_temp|precipitation|pressure|snow_depth|new_date  |
+-----------+--------+----------------+--------+---------+--------+-------------+--------+----------+----------+
|6.0        |3.5     |39.0            |7.5     |4.0      |0.8     |0.0          |102090.0|0.0       |2017-12-01|
|8.0        |0.0     |13.0            |10.4    |5.0      |2.5     |0.4          |102710.0|0.0       |2017-12-02|
|8.0        |0.0     |13.0            |10.0    |6.7      |3.0     |0.2          |102700.0|0.0       |2017-12-03|
|7.0        |0.0     |13.0            |9.4     |7.0      |4.0     |0.0          |103330.0|0.0       |2017-12-04|
|8.0        |0.0     |13.0            |11.6    |8.1      |6.8     |0.0          |103540.0|0.0       |2017-12-05|
|8.0        |0.0     |13.0            |12.5    |9.7      |7.7     |0.2          |102730.0|0.0   

In [16]:
df_wo_new.write.option("header","True").csv("generated")

AnalysisException: path file:/home/raddy/Projects/SparkProjects/generated already exists.

In [ ]:
def mapper(line):
    fields = line.split(',')
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), \
               age=int(fields[2]), num_friends=int(fields[3]))

23/03/25 08:26:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 30946984 ms exceeds timeout 120000 ms
23/03/25 08:26:37 WARN SparkContext: Killing executors is not supported by current scheduler.
23/03/25 08:26:38 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage